<a href="https://colab.research.google.com/github/Deepanshu41008/AVSE/blob/main/convolutional_decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#assuming Squeezeformer_output, shape [batch_size, time_steps, model_dim]

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_conv1d_decoder(model_dim=256, num_freq_bins=129):
    inputs = tf.keras.Input(shape=(None, model_dim))  # [batch, time, model_dim]

    x = layers.Conv1D(256, kernel_size=5, padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)

    x = layers.Conv1D(256, kernel_size=5, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv1D(128, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv1D(num_freq_bins, kernel_size=1, padding='same', activation='linear')(x)  # final output layer

    model = models.Model(inputs=inputs, outputs=x)
    model.compile(optimizer='adam', loss='mse')
    return model


In [ ]:

conv_decoder = build_conv1d_decoder(model_dim=256, num_freq_bins=129)
output_mag = conv_decoder(squeezeformer_output)


In [ ]:
import numpy as np
import librosa

# 1. Assume these are already available:
# enhanced_magnitude: shape [T, F] (i.e. [time frames, frequency bins])
# noisy_phase: shape [T, F], phase of the noisy STFT
# If enhanced_magnitude has shape [B, T, F], do enhanced_magnitude = enhanced_magnitude[0]

# 2. Reconstruct the complex-valued STFT
enhanced_stft = enhanced_magnitude * np.exp(1j * noisy_phase)

# Transpose if needed (librosa expects [freq_bins, time_frames])
enhanced_stft = enhanced_stft.T  # now shape [129, T]

# 3. Perform inverse STFT
window = scipy.signal.hamming(256, sym=False)
denoised_audio = librosa.istft(enhanced_stft, hop_length=64, win_length=256, window=window)

# 4. Save or play audio
import soundfile as sf
sf.write('output_denoised.wav', denoised_audio, samplerate=16000)
